# Урок: Рекуррентные нейронные сети

## Задание: 

* Попробуйте изменить параметры нейронной сети, генерирующей текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший текст из получившихся и опишите предпринятые для его получения действия. Можно использовать текст другого произведения

## Импорт библиотек

In [47]:
import numpy as np

import tensorflow.keras.utils as ku
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()
corpus = data.lower().split("\n")

--2021-08-10 12:05:33--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.76.128, 66.102.1.128, 172.253.120.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.76.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2021-08-10 12:05:33 (91.6 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [35]:
# Создаем объект для токенизации
tokenizer = Tokenizer(
  num_words=20000,
)
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [43]:
callback = EarlyStopping(monitor='val_loss', patience=10)

model = Sequential()
model.add(Embedding(total_words, 256, input_length=max_sequence_len-1))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 10, 256)           822016    
_________________________________________________________________
lstm_11 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 3211)              825227    
Total params: 2,172,555
Trainable params: 2,172,555
Non-trainable params: 0
_________________________________________________________________


In [44]:
 history = model.fit(
  predictors,
  label,
  batch_size=32,
  epochs=100,
  validation_split=0.2,
  callbacks=[callback],
  verbose=1
)

Epoch 1/100
387/387 [==============================] - 10s 20ms/step - loss: 6.8397 - accuracy: 0.0261 - val_loss: 6.7709 - val_accuracy: 0.0200
Epoch 2/100
387/387 [==============================] - 7s 19ms/step - loss: 6.4334 - accuracy: 0.0321 - val_loss: 6.8414 - val_accuracy: 0.0236
Epoch 3/100
387/387 [==============================] - 7s 19ms/step - loss: 6.2204 - accuracy: 0.0406 - val_loss: 6.8643 - val_accuracy: 0.0330
Epoch 4/100
387/387 [==============================] - 7s 19ms/step - loss: 5.9585 - accuracy: 0.0543 - val_loss: 6.9028 - val_accuracy: 0.0482
Epoch 5/100
387/387 [==============================] - 7s 18ms/step - loss: 5.6303 - accuracy: 0.0697 - val_loss: 7.0483 - val_accuracy: 0.0511
Epoch 6/100
387/387 [==============================] - 7s 18ms/step - loss: 5.2560 - accuracy: 0.0854 - val_loss: 7.1704 - val_accuracy: 0.0530
Epoch 7/100
387/387 [==============================] - 7s 19ms/step - loss: 4.8350 - accuracy: 0.1133 - val_loss: 7.3137 - val_accuracy

In [41]:
def generate_text(seed_text, next_words, model):
  # строим последовательность + новое предсказание  
  for _ in range(next_words):
    # текущая последовательность (токенизация и дополнение)
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
  
    output_word = ""
    # добавим новое слово в последовательность
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text += " " + output_word
  return seed_text

In [49]:
print (generate_text("united states", 10, model))
print (generate_text("india and china", 10, model))
print (generate_text("new york", 10, model))
print (generate_text("science and technology", 10, model))

united states with the world do i not glance more aside aside
india and china in their thoughts even it is it doth dwell in
new york to the world that i think his part ' in
science and technology in their thoughts even it is it doth dwell in
